In [1]:
# https://adrien.barbaresi.eu/blog/trafilatura-main-text-content-python.html

import requests
import bs4
from bs4 import BeautifulSoup
import os


# Get archives

In [2]:
page_url = "https://kghosh.substack.com/archive"

page_sourced = requests.get(page_url).content 

html_content = BeautifulSoup(page_sourced, "html.parser")
# #main > div.archive-page.typography.use-theme-bg > div > div > div.portable-archive-list > div:nth-child(19) > div.post-preview-content > a.post-preview-title.newsletter
links = html_content.findAll('a', class_="post-preview-title newsletter")
print(len(links),links) 
pages = [i.get('href') for i in links if (not (i.get('href') is None))]

pages = [ i for i in pages  if "https://kghosh.substack.com/p/" in i ]
pages = list(set([ i for i in pages  if not i.endswith("comments") ]))
len(pages), pages

12 [<a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20221210">20221210</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20221204">20221204</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20221127">20221127</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20221117">20221117</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20221106">20221106</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20221031">20221031</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20221016">20221016</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20221012">20221012</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20221002">20221002</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20220922">

(12,
 ['https://kghosh.substack.com/p/20220922',
  'https://kghosh.substack.com/p/20221031',
  'https://kghosh.substack.com/p/borges-babel-movies-a-short-story',
  'https://kghosh.substack.com/p/20221002',
  'https://kghosh.substack.com/p/20220917',
  'https://kghosh.substack.com/p/20221210',
  'https://kghosh.substack.com/p/20221012',
  'https://kghosh.substack.com/p/20221117',
  'https://kghosh.substack.com/p/20221204',
  'https://kghosh.substack.com/p/20221016',
  'https://kghosh.substack.com/p/20221127',
  'https://kghosh.substack.com/p/20221106'])

In [3]:
for page in pages:
    name = page.split("/")[-1]

    if not os.path.exists(".cache/"+name):
        page_sourced = requests.get(page).content 
        html_content = BeautifulSoup(page_sourced, "html.parser")
        content = html_content.findAll('div', class_="body markup")

        with open(".cache/"+name, 'w') as f:
            f.write(str(content))
        print(name,"saved")
    else:
        print(name,"exists.")

20220922 exists.
20221031 exists.
borges-babel-movies-a-short-story exists.
20221002 exists.
20220917 exists.
20221210 saved
20221012 exists.
20221117 exists.
20221204 exists.
20221016 exists.
20221127 exists.
20221106 exists.


In [4]:
import glob
import pandas as pd


cached_pages = glob.glob(".cache/*")


In [5]:
URLs = []
for page in cached_pages:
    with open(page) as fp:
        html_content = BeautifulSoup(fp, 'html.parser')  
    content = html_content.findAll('a')
    content = [i.get('href') for i in content if (not (i.get('href') is None))]
    for link in content:
        URLs.append([name,link])
    print(content) 

['https://www.jgballard.ca/non_fiction/jgb_fictions.html', 'https://hakaimagazine.com/features/greenlands-melting-ice-sheet-brings-an-unexpected-flow-of-wealth-potential/?utm_source=substack&utm_medium=email', 'https://www.theverge.com/2022/10/25/23422359/shutterstock-ai-generated-art-openai-dall-e-partnership-contributors-fund-reimbursement', 'https://www.axa.com/en/magazine/2022-future-risks-report?utm_source=substack&utm_medium=email', 'https://www.rentcafe.com/blog/rental-market/market-snapshots/adaptive-reuse-apartments/?utm_source=newsletter&utm_medium=email&utm_campaign=newsletter_axioswhatsnext&stream=science', 'https://wordcraft-writers-workshop.appspot.com/learn?utm_source=substack&utm_medium=email', 'https://wordcraft-writers-workshop.appspot.com/stories/ken-liu', 'https://seenapse.it/?utm_source=substack&utm_medium=email', 'https://dl.acm.org/doi/pdf/10.1145/3564017', 'https://futurescanvas.com/en', 'https://modemworks.com/research/patented-futures/?utm_source=substack&utm_

['https://www.youtube.com/watch?v=qDXqj-zByOo', 'https://www.paris.fr/pages/la-petite-ceinture-et-ses-promenades-ecologiques-7855?utm_source=substack&utm_medium=email', 'https://umdearborn.edu/news/ev-charging-stations-could-be-target-hackers?utm_source=newsletter&utm_medium=email&utm_campaign=newsletter_axioscodebook&stream=top', 'https://interestingengineering.com/innovation/groundbreaking-motionless-wind-turbine', 'https://www.fastcompany.com/90780072/affordable-apartments-passive-house-standardsare-designed-to-use-almost-no-energy?utm_source=substack&utm_medium=email', 'https://asia.nikkei.com/Spotlight/Datawatch/More-empty-homes-Japan-s-housing-glut-to-hit-10m-in-2023', 'https://www.theverge.com/22807858/tiktok-influencer-microsoft-excel-instagram-decoder-podcast?utm_source=substack&utm_medium=email', 'https://digitalnative.substack.com/p/the-rise-of-the-digitally-native?utm_source=post-email-title&publication_id=38500&post_id=81595462&isFreemail=true&token=eyJ1c2VyX2lkIjo4NTkxMDY

['https://news.ycombinator.com/item?id=32611247', 'https://peabee.substack.com/p/17-one-kitchen-hundreds-of-internet', 'https://www.chemistryworld.com/podcasts/low-background-steel/3009874.article', 'https://bigthink.com/strange-maps/derinkuyu-underground-city/', 'https://psychedelicspotlight.com/do-you-even-trip-bro-on-psychedelic-masculinity/', 'https://www.techspot.com/news/95774-end-moore-law-forced-youtube-make-own-video.html', 'https://www.marindie.com/digital-products/', 'https://opg.optica.org/boe/fulltext.cfm?uri=boe-13-9-4684&id=491326', 'https://freedom-to-tinker.com/2022/08/03/the-anomaly-of-cheap-complexity/', 'https://www.theatlantic.com/culture/archive/2021/06/information-efficiency-history-filing-cabinet/619106/?utm_source=substack&utm_medium=email', 'https://www.federalreserve.gov/econres/notes/feds-notes/long-covid-cognitive-impairment-and-the-stalled-decline-in-disability-rates-20220805.htm', 'https://knowablemagazine.org/article/health-disease/2022/long-covid-parall

In [6]:
import hashlib

df = pd.DataFrame(URLs)
df.columns = ["page","url"]
df = df.drop_duplicates(subset=["url"])
df["url"] = df.url.apply(lambda x: str(x).encode('utf-8'))

df["hash"] = df.url.apply(lambda x: hashlib.md5(str(x).encode('utf-8')).hexdigest())
df.to_parquet("urls.parquet.gzip",  engine='pyarrow', compression='gzip')
df

,page,url,hash
0,20221106,b'https://www.jgballard.ca/non_fiction/jgb_fic...,8ebb431ce8d949d5a1ecb52bbd42a69f
1,20221106,b'https://hakaimagazine.com/features/greenland...,6c6a5e8d50e3045323cf202ba5b17ea9
2,20221106,b'https://www.theverge.com/2022/10/25/23422359...,99ffe526026a0494fc25d44aba64bd70
3,20221106,b'https://www.axa.com/en/magazine/2022-future-...,792ea5817e7c3331760ac18e3090869b
4,20221106,b'https://www.rentcafe.com/blog/rental-market/...,8bff520dc945204951b32a6c4b10571e
...,...,...,...
486,20221106,b'https://knowablemagazine.org/article/health-...,c47cb1fe2b4ae786edcfe793f7c528c2
487,20221106,b'https://blog.seekwell.io/gpt3',492f4356567de26e0afe1f008454e899
489,20221106,"b'https://substackcdn.com/image/fetch/f_auto,q...",07a77cb65e9b99211c078e05f90f6eca
490,20221106,b'https://en.wikipedia.org/wiki/Valeriepieris_...,6ab95ef7d3a59b67b7851af561ed903b
